In [7]:
from data import *
from vf import *

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import lightning as pl
from torchdiffeq import odeint_adjoint as odeint
import numpy as np
import random


SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
train_traj, test_traj = get_dataset(sdt=['rotationRate'], act_labels=['jog'])

[INFO] -- Selected sensor data types: ['rotationRate']
[INFO] -- Selected activites: ['jog']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(134231, 10)


In [8]:
class TrajectoryDataset(Dataset):
    def __init__(self, dataframe, max_horizon=100):
        self.data = dataframe
        self.trajectories = []
        self.max_horizon = max_horizon
        self.feature_columns = dataframe.columns.difference(['exact_id', 'timeidx']).tolist()
        
        for exact_id, group in self.data.groupby('exact_id'):
            group_sorted = group.sort_values('timeidx')
            full_traj = group_sorted[self.feature_columns].values.astype(float)
        
            n_segments = len(full_traj) // self.max_horizon
            for i in range(n_segments):
                start_idx = i * self.max_horizon
                end_idx = start_idx + self.max_horizon
                segment = full_traj[start_idx:end_idx]
                self.trajectories.append(segment)

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.trajectories[idx])

def single_collate(batch):
    return batch[0]

In [ ]:
dataset = TrajectoryDataset(train_traj)
dataloader = DataLoader(
    dataset,
    batch_size=20,
    shuffle=True,
    collate_fn=single_collate
)
    
model = DynamicSystemLearner(dt=0.01, lr=1e-3)


trainer = pl.Trainer(
    max_epochs=20,
    accelerator="auto",
    devices=1,
    gradient_clip_val=0.5,
    enable_progress_bar=True,
    check_val_every_n_epoch=1
)


trainer.fit(model, dataloader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | vf        | NodeField | 27     | train
1 | criterion | MSELoss   | 0      | train
------------------------------------------------
27        Trainable params
0         Non-trainable params
27        Total params
0.000     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode
/home/ivan/Python/m1p/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/ivan/Python/m1p/l

Training: |                                               | 0/? [00:00<?, ?it/s]

torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([1, 1]) torch.Size([1, 2])
torch.Size([

In [ ]:
for x in dataloader:
    break

In [4]:
for batch in dataloader:
    break

In [7]:
batch.size(0)

100

In [10]:
np.min(train_traj['rotationRate.x'])

np.float64(-11.135991)